In [1]:
import sys
sys.path.append('../')

from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.smore_datareader import SmoreDataReader
from preprocessing.xgboost_transformer import  XGBoostTransformer
import numpy 

user_path = './ml-100k/u.user'
item_path = './ml-100k/u.item'
user_item = './ml-100k/u.data'
reader = IMDBDataReader()

user_item  = reader.read_user_item_rating(user_item)
users = reader.read_user_data(user_path)
items = reader.read_item_data(item_path)


train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]

transformer = XGBoostTransformer(users, items, train_user_item)
smorereader = SmoreDataReader(transformer.u_idx, transformer.i_idx, "rep_dw.txt")
smore_user_vector = smorereader.read_user_data()

total_users = {} 
for k, v in users.items(): 
    if k in smore_user_vector: 
        total_users[k] = v+smore_user_vector[k]

# transformer = XGBoostTransformer(total_users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(total_users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(total_users, items, test_user_item)

In [2]:
X_train[:10]

[[(0, 21),
  (2, 1),
  (21, 1),
  (24, -0.269325),
  (25, 0.0617571),
  (26, -0.168259),
  (27, -0.385321),
  (28, 0.0360274),
  (29, -0.223947),
  (30, 0.0230822),
  (31, -0.0654962),
  (32, 0.0270713),
  (33, 0.0463822),
  (34, 0.0612527),
  (35, -0.0193315),
  (36, 0.211944),
  (37, 0.0202455),
  (38, -0.362862),
  (39, -0.205064),
  (40, -0.0368804),
  (41, -0.126533),
  (42, 0.115612),
  (43, -0.0388958),
  (44, 0.188789),
  (45, -0.0923134),
  (46, 0.0716938),
  (47, -0.41043),
  (48, -0.00857141),
  (49, 0.105059),
  (50, 0.115862),
  (51, 0.0014072),
  (52, -0.244144),
  (53, 0.169029),
  (54, -0.0635373),
  (55, -0.176073),
  (56, -0.170946),
  (57, 0.131099),
  (58, 0.16397),
  (59, 0.148741),
  (60, 0.209373),
  (61, 0.0945409),
  (62, -0.127754),
  (63, -0.131902),
  (64, 0.150297),
  (65, -0.163779),
  (66, 0.217329),
  (67, 0.080453),
  (68, -0.0899917),
  (69, -0.131759),
  (70, -0.0761843),
  (71, -0.258111),
  (72, -0.0310267),
  (73, -0.435242),
  (74, -0.190127),
  (

In [ ]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2

In [3]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/xgboost-movielens'
train_key      = 'train'
test_key       = 'test'
test_cold_key  = 'test_cold'


import io,boto3
import sagemaker.amazon.common as smac

def writeToLocalCSV(X, key, Y): 
    f_name = key + ".csv"
    output = open(f_name, 'w')

    for i in range(0, len(X)): 
        line = ''
        raw = X[i]
        line += str(int(Y[i])) 
        line += ' '
        for ele in raw:
            line += str(ele[0])+':'+str(float(ele[1]))+' '
        line += '\n'
        output.write(line)    
    return f_name  


 

train_data = writeToLocalCSV(X_train, train_key, Y_train)    
test_data = writeToLocalCSV(X_test, test_key, Y_test)    
test_cold_data = writeToLocalCSV(X_cold_test, test_cold_key, Y_cold_test)    


In [4]:
!pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
import xgboost as xgb
dtrain = xgb.DMatrix('train.csv')
test = xgb.DMatrix('test.csv')
test_cold = xgb.DMatrix('test_cold.csv')



param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'alpha':10,
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 3000  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)


preds = bst.predict(dtrain)


[09:30:02] 80000x108 matrix with 5592496 entries loaded from train.csv
[09:30:02] 2863x108 matrix with 199272 entries loaded from test.csv
[09:30:02] 89x108 matrix with 6262 entries loaded from test_cold.csv
[09:30:02] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
from sklearn.metrics import accuracy_score

preds = bst.predict(test)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)


In [ ]:
accuracy = accuracy_score(Y_test, predictions)
accuracy

In [ ]:

preds = bst.predict(test_cold)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)

In [ ]:
accuracy = accuracy_score(Y_cold_test, predictions)
accuracy